In [10]:
import os
import sys 
import subprocess
from itertools import product

def make_file(embeddings_dim,hidden_dim,RNN_models,GNN_models,n_layers,upsample,lr,base):

    fs = []
    for comb in product(*[embeddings_dim,hidden_dim,RNN_models,GNN_models,n_layers,upsample,lr]):
            emb_dim = comb[0]
            hid_dim = comb[1]
            rnn = comb[2]
            gnn = comb[3]
            layers = comb[4]
            upsamp = comb[5]
            lr = comb[6]
            file = base + f'/{rnn}_{gnn}_layers_{layers}_emb_{emb_dim}_hidd_{hid_dim}_lr_{lr}_samp_rate_{upsamp}.sh'
            name = f'{rnn}_{gnn}_layers_{layers}_emb_{emb_dim}_hidd_{hid_dim}_lr_{lr}_samp_rate_{upsamp}'
            fs.append(file)
            # print(file)
            with open(file,'w+') as f :
                # CHANGE PATHS
                f.write(\
f'''#!/bin/bash
#SBATCH --account=rrg-cbravo
#SBATCH --mem-per-cpu=100G      # increase as needed
#SBATCH --cpus-per-task=2
#SBATCH --time=20:00:00
#SBATCH --output /home/emiliano/projects/def-cbravo/emiliano/RappiAnomalyDetection/logs/final_models/{name}.out # STDOUT

module load python/3.8.10

source $HOME/TGN/bin/activate
cd /home/emiliano/projects/def-cbravo/emiliano/RappiAnomalyDetection/methods/GAT

python gru.py --GNN {gnn} --RNN {rnn}  --run {name} --hidden_dim {hid_dim} --lr {lr} --embedding_dim {emb_dim} --GNN_layers {layers} --epochs 300 --sample_rate {upsamp} -val_h0_test --boot_sample 10000

                                ''')
    return fs
def run_sbatch(files):
    for f in files: 
        subprocess.call(['sbatch',f])

In [11]:

embeddings_dim = [200,300]
hidden_dim = [200,300]
RNN_models = ['GRU','LSTM']
GNN_models = ['GCN','GAT']
n_layers = [1,3]
upsample = [0,.75]
lr = [.0001 ]
#change path
base = '/home/emiliano/projects/def-cbravo/emiliano/RappiAnomalyDetection/execution_scripts/final_models'
fs = make_file(embeddings_dim,hidden_dim,RNN_models,GNN_models,n_layers,upsample, lr, base = base )
run_sbatch(fs)

Submitted batch job 65290999
Submitted batch job 65291000
Submitted batch job 65291001
Submitted batch job 65291002
Submitted batch job 65291003
Submitted batch job 65291004
Submitted batch job 65291005
Submitted batch job 65291006
Submitted batch job 65291007
Submitted batch job 65291008
Submitted batch job 65291009
Submitted batch job 65291010
Submitted batch job 65291011
Submitted batch job 65291012
Submitted batch job 65291013
Submitted batch job 65291014
Submitted batch job 65291015
Submitted batch job 65291016
Submitted batch job 65291017
Submitted batch job 65291018
Submitted batch job 65291019
Submitted batch job 65291020
Submitted batch job 65291021
Submitted batch job 65291022
Submitted batch job 65291023
Submitted batch job 65291024
Submitted batch job 65291025
Submitted batch job 65291026
Submitted batch job 65291027
Submitted batch job 65291028
Submitted batch job 65291029
Submitted batch job 65291030
Submitted batch job 65291031
Submitted batch job 65291032
Submitted batc

In [12]:
import os
import sys 
import subprocess
from itertools import product

def make_file_simple(embeddings_dim,hidden_dim,upsample,only_models,base):
    configs = {}
    fs = []
    for comb in product(*[embeddings_dim,hidden_dim,upsample,only_models]):
            emb_dim = comb[0]
            hid_dim = comb[1]
            upsamp = comb[2]
            onlies = comb[3]
            onlies_names = onlies.replace(" " ,"_")
            onlies_names = onlies.replace("-" ,"_")
            # print(onlies)
            if '-ONLY_GNN' in onlies:
                layers = onlies[-1]
            else: 
                layers = ''
                
            file = base + f'/{onlies_names}_layers_{layers}_emb_{emb_dim}_hidd_{hid_dim}_{upsamp}.sh'

            name = f'{onlies_names}_layers_{layers}_emb_{emb_dim}_hidd_{hid_dim}_sample_rate_{upsamp}_low_lr'.replace(" " ,"_")
            # print(f'python gru_simple_models.py --GNN {gnn} --RNN {rnn}  --run {name.replace(" " ,"_")} --hidden_dim {hid_dim} --embedding_dim {emb_dim} --GNN_layers {layers} --epochs 100 {onlies} {upsamp} {v_h0}')
            # print(name)
            # raise Exception
            fs.append(file)
            # print(file)
            with open(file,'w+') as f :
                f.write(\
f'''#!/bin/bash
#SBATCH --account=rrg-cbravo
#SBATCH --mem-per-cpu=100G      # increase as needed
#SBATCH --cpus-per-task=2
#SBATCH --time=15:00:00
#SBATCH --output /home/emiliano/projects/def-cbravo/emiliano/RappiAnomalyDetection/logs/lower_lr/{name}.out # STDOUT

module load python/3.8.10


source $HOME/TGN/bin/activate
cd /home/emiliano/projects/def-cbravo/emiliano/RappiAnomalyDetection/methods/GAT

python gru_simple_models.py  --run {name} --lr .00001 --hidden_dim {hid_dim} --embedding_dim {emb_dim} --epochs 300 {onlies} --sample_rate {upsamp} -val_h0_test

                                ''')
    return fs
def run_sbatch_simple(files):
    for f in files: 
        subprocess.call(['sbatch',f])




In [13]:
base = '/home/emiliano/projects/def-cbravo/emiliano/RappiAnomalyDetection/execution_scripts/final_models'

embeddings_dim = [200,300]
hidden_dim = [200,300]
# RNN_models = ['GRU','LSTM']
# GNN_models = ['GCN','GAT']
page_rank_only_models = [ '-ONLY_RNN -PAGE_RANK --RNN GRU ','-ONLY_RNN -PAGE_RANK --RNN LSTM']
only_models=['-ONLY_RNN --RNN GRU', '-ONLY_RNN --RNN LSTM' , '-ONLY_GNN --GNN GAT --GNN_layers 3' ,
     '-ONLY_GNN --GNN GCN --GNN_layers 3','-ONLY_GNN --GNN GAT --GNN_layers 1 --early_stop 50' ,
     '-ONLY_GNN --GNN GCN --GNN_layers 1 --early_stop 50']
# n_layers = [1,3]
upsample = [0,.75]

# embeddings_dim = [3]
# hidden_dim = [3]

# page_rank_only_models = [ '-ONLY_RNN -PAGE_RANK --RNN GRU','-ONLY_RNN -PAGE_RANK --RNN LSTM']
# only_models=['-ONLY_RNN --RNN GRU', '-ONLY_RNN --RNN LSTM' , '-ONLY_GNN --GNN GAT' , '-ONLY_GNN --GNN GCN']
# n_layers = [1]
# upsample = [0,.5]

fs = make_file_simple(embeddings_dim,hidden_dim,upsample,only_models, base = base )
run_sbatch_simple(fs)

fs = make_file_simple(embeddings_dim,hidden_dim,upsample,page_rank_only_models, base = base )
run_sbatch_simple(fs)


Submitted batch job 65291063
Submitted batch job 65291064
Submitted batch job 65291065
Submitted batch job 65291066
Submitted batch job 65291067
Submitted batch job 65291068
Submitted batch job 65291069
Submitted batch job 65291070
Submitted batch job 65291071
Submitted batch job 65291072
Submitted batch job 65291073
Submitted batch job 65291074
Submitted batch job 65291075
Submitted batch job 65291076
Submitted batch job 65291077
Submitted batch job 65291078
Submitted batch job 65291079
Submitted batch job 65291080
Submitted batch job 65291081
Submitted batch job 65291082
Submitted batch job 65291083
Submitted batch job 65291084
Submitted batch job 65291085
Submitted batch job 65291086
Submitted batch job 65291087
Submitted batch job 65291088
Submitted batch job 65291089
Submitted batch job 65291090
Submitted batch job 65291091
Submitted batch job 65291092
Submitted batch job 65291093
Submitted batch job 65291094
Submitted batch job 65291095
Submitted batch job 65291096
Submitted batc

In [1]:
s  = 'model,train_time,VAL_AUC_SEEN,VAL_AUC_UNSEEN,VAL_AUPRC_SEEN,VAL_AUPRC_UNSEEN,VAL_BOOT_SEEN_AUC_MEAN,VAL_BOOT_SEEN_AUC_STD,VAL_BOOT_SEEN_AUC_Q0,VAL_BOOT_SEEN_AUC_Q25,VAL_BOOT_SEEN_AUC_Q50,VAL_BOOT_SEEN_AUC_Q75,VAL_BOOT_SEEN_AUC_Q100,VAL_BOOT_UNSEEN_SEEN_AUC_MEAN,VAL_BOOT_UNSEEN_SEEN_AUC_STD,VAL_BOOT_UNSEEN_SEEN_AUC_Q0,VAL_BOOT_UNSEEN_SEEN_AUC_Q25,VAL_BOOT_UNSEEN_SEEN_AUC_Q50,VAL_BOOT_UNSEEN_SEEN_AUC_Q75,VAL_BOOT_UNSEEN_SEEN_AUC_Q100,TEST_BOOT_SEEN_AUC_MEAN,TEST_BOOT_SEEN_AUC_STD,TEST_BOOT_SEEN_AUC_Q0,TEST_BOOT_SEEN_AUC_Q25,TEST_BOOT_SEEN_AUC_Q50,TEST_BOOT_SEEN_AUC_Q75,TEST_BOOT_SEEN_AUC_Q100,TEST_BOOT_UNSEEN_SEEN_AUC_MEAN,TEST_BOOT_UNSEEN_SEEN_AUC_STD,TEST_BOOT_UNSEEN_SEEN_AUC_Q0,TEST_BOOT_UNSEEN_SEEN_AUC_Q25,TEST_BOOT_UNSEEN_SEEN_AUC_Q50,TEST_BOOT_UNSEEN_SEEN_AUC_Q75,TEST_BOOT_UNSEEN_SEEN_AUC_Q100,TEST_AUC_SEEN,TEST_AUC_UNSEEN,TEST_AUPRC_SEEN,TEST_AUPRC_UNSEEN,TRAIN_AUC,TRAIN_AUPRC'
s.split(',')

['model',
 'train_time',
 'VAL_AUC_SEEN',
 'VAL_AUC_UNSEEN',
 'VAL_AUPRC_SEEN',
 'VAL_AUPRC_UNSEEN',
 'VAL_BOOT_SEEN_AUC_MEAN',
 'VAL_BOOT_SEEN_AUC_STD',
 'VAL_BOOT_SEEN_AUC_Q0',
 'VAL_BOOT_SEEN_AUC_Q25',
 'VAL_BOOT_SEEN_AUC_Q50',
 'VAL_BOOT_SEEN_AUC_Q75',
 'VAL_BOOT_SEEN_AUC_Q100',
 'VAL_BOOT_UNSEEN_SEEN_AUC_MEAN',
 'VAL_BOOT_UNSEEN_SEEN_AUC_STD',
 'VAL_BOOT_UNSEEN_SEEN_AUC_Q0',
 'VAL_BOOT_UNSEEN_SEEN_AUC_Q25',
 'VAL_BOOT_UNSEEN_SEEN_AUC_Q50',
 'VAL_BOOT_UNSEEN_SEEN_AUC_Q75',
 'VAL_BOOT_UNSEEN_SEEN_AUC_Q100',
 'TEST_BOOT_SEEN_AUC_MEAN',
 'TEST_BOOT_SEEN_AUC_STD',
 'TEST_BOOT_SEEN_AUC_Q0',
 'TEST_BOOT_SEEN_AUC_Q25',
 'TEST_BOOT_SEEN_AUC_Q50',
 'TEST_BOOT_SEEN_AUC_Q75',
 'TEST_BOOT_SEEN_AUC_Q100',
 'TEST_BOOT_UNSEEN_SEEN_AUC_MEAN',
 'TEST_BOOT_UNSEEN_SEEN_AUC_STD',
 'TEST_BOOT_UNSEEN_SEEN_AUC_Q0',
 'TEST_BOOT_UNSEEN_SEEN_AUC_Q25',
 'TEST_BOOT_UNSEEN_SEEN_AUC_Q50',
 'TEST_BOOT_UNSEEN_SEEN_AUC_Q75',
 'TEST_BOOT_UNSEEN_SEEN_AUC_Q100',
 'TEST_AUC_SEEN',
 'TEST_AUC_UNSEEN',
 'TEST_AUPRC_SEEN

In [23]:
import os
from tqdm import tqdm
b = '/home/emiliano/projects/def-cbravo/emiliano/RappiAnomalyDetection/logs/final_models/'
counter = 0 
files =  os.listdir(b)
for f in tqdm(files): 

    with open(b + f, 'r') as f:
        lines = f.read().splitlines()
        last_line = lines[-50:]
        last_line = ''.join(last_line)
        keyword_list = ['error', 'exception', 'argument']

        if any(word in last_line for word in keyword_list):
            counter += 1 
            # print ('found one of em',f)
            # print(last_line)
print(counter)



100%|██████████| 320/320 [00:00<00:00, 886.52it/s]

14
